In [35]:
import pandas as pd
import plotly.express as px

coral = pd.read_csv(
    "../../data/processed/CORIS_DATA/ATLANTIC/PUERTO_RICO/Benthic_Cover_Puerto_Rico_Cleaned.csv"
)

In [36]:
coral.columns
print(sorted(coral["YEAR"].unique()))

[np.int64(-1), np.int64(2014), np.int64(2016), np.int64(2017), np.int64(2019), np.int64(2021), np.int64(2023)]


In [37]:
# --- clean + prep ---
d = coral.copy()

YEAR = 2023
d = d[d["YEAR"] == YEAR]

d["latitude"] = pd.to_numeric(d["latitude"], errors="coerce")
d["longitude"] = pd.to_numeric(d["longitude"], errors="coerce")
d["HARDBOTTOM_P"] = pd.to_numeric(d["HARDBOTTOM_P"], errors="coerce")

# drop rows without spatial data or percentage
d = d.dropna(subset=["latitude", "longitude", "HARDBOTTOM_P"])

# wrap longitudes if needed (safe even if already -180..180)
d["longitude"] = ((d["longitude"] + 180) % 360) - 180

In [38]:

# --- plot ---
fig = px.scatter_geo(
    d,
    lat="latitude",
    lon="longitude",
    color="HARDBOTTOM_P",
    color_continuous_scale="Viridis",   # good perceptual gradient
    range_color=(0, 100),                # percent scale (adjust if needed)
    opacity=0.7,
    hover_data={
        "HARDBOTTOM_P": True,
        "SOFTBOTTOM_P": True,
        "RUBBLE_P": True,
        "HABITAT_CD": True,
        "YEAR": True,
        "MIN_DEPTH": True,
        "MAX_DEPTH": True,
    },
)

fig.update_geos(
    projection_type="natural earth",
    showland=True,
    landcolor="rgb(220,220,220)",
    showocean=True,
    oceancolor="rgb(200,220,255)",
    showcountries=True,
)

fig.update_layout(
    title="Hardbottom Coverage (%)",
    height=700,
    margin=dict(l=0, r=0, t=50, b=0),
    coloraxis_colorbar=dict(
        title="Hardbottom (%)"
    ),
)

fig.show()